In [1]:
!git clone https://github.com/krishnaik06/SpamClassifier.git

Cloning into 'SpamClassifier'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
data = pd.read_csv('/content/SpamClassifier/smsspamcollection/train', sep='\t',names=['label', 'message'])

In [4]:
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data['spam'] = labelencoder.fit_transform(data['label'])

In [6]:
data.head()

,label,message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [7]:
data.drop(['label'],inplace=True, axis=1)

In [8]:
import re
ps = PorterStemmer()
wnl = WordNetLemmatizer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
corpus = []
for message in data['message']:
  final_msg = re.sub('[^a-zA-Z]', ' ', message)
  final_msg = final_msg.lower()
  final_msg = final_msg.split()
  final_msg = [ps.stem(word) for word in final_msg if not word in stopwords.words('english')]
  final_msg = ' '.join(final_msg)
  corpus.append(final_msg)
print(corpus[0:5])

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat', 'ok lar joke wif u oni', 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli', 'u dun say earli hor u c alreadi say', 'nah think goe usf live around though']


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
X = CV.fit_transform(corpus)
y = data['spam']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [12]:
print(X_train.shape,'   ', X_test.shape,'   ', y_train.shape,'   ', y_test.shape)

(4457, 6296)     (1115, 6296)     (4457,)     (1115,)


In [13]:
from sklearn.naive_bayes import MultinomialNB
spam_detector = MultinomialNB()
spam_detector.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
y_pred = spam_detector.predict(X_test)
#Accuracy_score = spam_detector.score(y_test, y_pred)

In [15]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
print(acc)

0.9668161434977578


In [ ]:
import pickle
file = open('SpamDetector.pkl', 'wb')
pickle.dump(spam_detector, file)

In [18]:
import pickle
file = open('transform.pkl', 'wb')
pickle.dump(CV, file)